# Descision Tree Classifier

In descision Trees we try to make yes/no descisions at each node of the tree and based on the threshold and feature we either move left or right. After reaching the end (leaf node) majority class in that node will be the answer.

## Measures for splitting
Each Node of the tree represents group of data, and it has some impurity value. Impurity is nothing but measure of mixedness/randomness in data.
Commonly used measures of calculating impurity are Gini and Entropy.

### Gini Index
$ I_{G} = 1-\sum_{i=1}^{k} P(k)^{2} $

For example,  
$ class = [1,1,1], I_{G}= 0 $ (zero means pure)  
$ class = [1,1,0,0], I_{G}= 0.5 $ (Max for binary classification)   
$ class = [1,1,1,0], I_{G}= 0.375 $  


### Entropy
$ I_{E} = -\sum_{i=1}^{k} P(k)\log_{2} {P(k)} $

For example,  
$ class = [1,1,1], I_{E}= 0 $ (zero means pure)  
$ class = [1,1,0,0], I_{E}= 1 $ (Max impure or equal classes)   
$ class = [1,1,1,0], I_{E}= 0.811 $  

### Weighted Impurity
Weighted impurity is the total proportionally summed impurity of children of a Node.  
Suppose for root node, two splits occur(that is two child nodes are formed), and the impurity of each child node is I1 and I2, then 
   
$ WI = \dfrac{sample\_count_{1}}{parent\_sample\_count} * I1 + \dfrac{sample\_count_{2}}{parent\_sample\_count} * I2 $

### Gain (Information Gain)

Information Gain shows how much better the split has happend for a parent for that particular split.

IG(of a split) = Parent_entropy - Weighted_impurity(of that split)  
  
Gain is used to decide the feature and threshold of each node, the max the gain the better the split

In [21]:
import numpy as np
class Node:
    def __init__(self,* , left=None, right=None, feature=None, threshold=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.right = right
        self.left = left
        self.value = value
    
    def _is_leaf(self):
        return self.value is not None
    
class DescisionTreeClassifier:
    def __init__(self, max_depth=5, min_samples_split=2, criterion = "gini"):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.root = None
    
    def _gini(self,y):
        _ , count = np.unique(y, return_counts=True)
        total = np.sum(count)
        probsquare_array = [(c/total)**2 for c in count]
        return 1-np.sum(probsquare_array)

    def _entropy(self, y):
        _ , count = np.unique(y, return_counts=True)
        total = np.sum(count)
        prob_array = [(c/total) for c in count]
        return -1*np.sum([p*np.log2(p) for p in prob_array])
    
    def impurity(self, y):
        if self.criterion=="gini":
            return self._gini(y)
        else:
            return self._entropy(y)
        
    def _best_split(self, X,Y):
        n_samples, n_features = X.shape
        if n_samples <= 1:
            return None, None, None, None, None
        
        parent_imp = self.impurity(Y)
        gain = 0
        threshold = 0
        feature = None
        right = None
        left = None

        for current_feature in range(n_features):
            sorted_fea = np.argsort(X[:,current_feature]) #So that while sorting both X and y ,order matches
            X_sorted, Y_sorted = X[sorted_fea, current_feature], Y[sorted_fea]
            for i in range(1,n_samples):
                if X_sorted[i]==X_sorted[i-1]:
                    continue
                current_threshold = (X_sorted[i]+X_sorted[i-1])/2
                current_left_idx = sorted_fea[:i]
                current_right_idx = sorted_fea[i:]

                if len(current_left_idx)==0 or len(current_right_idx)==0:
                    continue
                left_imp = self.impurity(Y[current_left_idx])
                right_imp = self.impurity(Y[current_right_idx])
                weighted_imp = (len(current_left_idx)*left_imp+len(current_right_idx)*right_imp)/n_samples
                current_gain = parent_imp-weighted_imp
                if current_gain>gain:
                    gain = current_gain
                    threshold = current_threshold
                    feature = current_feature
                    right = current_right_idx
                    left = current_left_idx

        return gain, threshold, feature, left, right
    
    def _build_tree(self, X, Y, depth = 0):
        values, count = np.unique(Y, return_counts=True)
        if depth>=self.max_depth or len(values)==1 or len(Y)<self.min_samples_split:
            leaf_value = values[np.argmax(count)]
            return Node(value=leaf_value)
        gain, th_hold, fea, leftid, rightid = self._best_split(X,Y)
        if gain==0 or leftid is None or rightid is None:
            leaf_value = values[np.argmax(count)]
            return Node(value=leaf_value)
        left = self._build_tree(X[leftid], Y[leftid], depth+1)
        right = self._build_tree(X[rightid], Y[rightid], depth+1)
        return Node(left=left, right=right, feature=fea, threshold=th_hold)

    def fit(self, X, Y):
        self.root = self._build_tree(X,Y)

    def _predict_one(self, X, node: Node):
        if node._is_leaf():
            return node.value
        if X[node.feature]<=node.threshold:
            return self._predict_one(X, node.left)
        else:
            return self._predict_one(X, node.right)

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])
        


In [22]:
import pandas as pd
df = pd.read_csv("D:/Ml/part1/data/diabetes.csv")
X = df.drop(columns=["Outcome"]).values
Y = df['Outcome'].values
type(X)

numpy.ndarray

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2, random_state=42)

In [43]:
model = DescisionTreeClassifier(max_depth=3)
model.fit(X_train, Y_train)

In [44]:
#calculating predicted values
Y_pred = model.predict(X_test)
Y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.7597402597402597

In [46]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred)

array([[83, 16],
       [21, 34]])